# SPRINT 10 - Web scraping i automatització

Gran parte de los datos que podemos encontrar para analizar utilizando el aprendizaje automático es texto libre. En este sprint veremos cómo analizarlo, limpiarlo y realizar análisis de sentimientos.


### EXERCICI 1

Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 



### ¿Qué es Web Scraping?

El web scraping es una técnica utilizada para extraer información de páginas web de forma automatizada. Es una forma de recopilar datos de una página web de manera programática y estructurada, lo que permite analizar y utilizar esta información de manera eficiente.

El web scraping se realiza mediante el uso de un programa o un script que explora una página web, lee y analiza su código HTML, y extrae la información relevante que se desea obtener. Por ejemplo, se puede utilizar web scraping para recopilar información de precios de productos en un sitio de comercio electrónico, obtener los datos de contacto de una empresa en su página web, o extraer noticias de un sitio de noticias en línea.

Para realizar el web scraping, se utilizan herramientas y librerías de programación, como Python, Beautiful Soup, Scrapy, entre otras, que permiten acceder al contenido de una página web y extraer la información relevante.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import nltk
nltk.download('punkt') #se utiliza para tokenizar frases en lenguaje natural
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4') 
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon') #Analysis with Vader

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Realizaremos primero con la web https://quotes.toscrape.com/ y luego con https://www.bolsasymercados.es/bme-exchange/es/Indices/Ibex .

Utilizaremos Beautiful Soup y luego Selenium

**Beautiful Soup**

Beautiful Soup es una biblioteca de Python utilizada para analizar y extraer información de archivos HTML y XML. La biblioteca proporciona una forma sencilla de buscar y manipular la estructura del documento HTML o XML, lo que la hace muy útil para el análisis de páginas web, el scraping de datos y la extracción de información en proyectos de minería de datos.

Definiremos las dos url: 'url' y 'url2', cada una correspondiente a la web con la que queremos trabajar y realizar la extracción de datos

In [2]:
import requests
from bs4 import BeautifulSoup
import re

url = "http://quotes.toscrape.com"
url2 = "https://es.wikipedia.org/wiki/Anexo:Aglomeraciones_urbanas_m%C3%A1s_pobladas_del_mundo"

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

#Encontrar elementos
quote_elements = soup.find_all("div", class_="quote")
quote_elements


[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag/world/page/1/">world</a>
 </div>
 </div>,
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
 <span>by <small class="author" itempr

Como la página es de citas, vemos como se divide en autor y la cita.
Ahora procedemos a dividir ambas y transformarlo a un Data Frame:

In [3]:
df = pd.DataFrame()
for quote_element in quote_elements:
    quote = quote_element.find("span", class_="text")
    author = quote_element.find("small", class_="author")
    df = df.append({'Quote' : quote.text.strip(),
                        'Author' : author.text.strip()},
                    ignore_index = True)
    
df

<ipython-input-3-66c2ceb5aafd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Quote' : quote.text.strip(),
<ipython-input-3-66c2ceb5aafd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Quote' : quote.text.strip(),
<ipython-input-3-66c2ceb5aafd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Quote' : quote.text.strip(),
<ipython-input-3-66c2ceb5aafd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Quote' : quote.text.strip(),
<ipython-input-3-66c2ceb5aafd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

,Quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


In [22]:
df.to_csv('df.csv', index=False)

Podemos observar como tenemos una columna con la frase y otra con el autor de la misma.

Continuamos realizándolo con la web de Wikipedia:

In [4]:
page = requests.get(url2)

soup2 = BeautifulSoup(page.content, "html.parser")


In [5]:
soup2

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Anexo:Aglomeraciones urbanas más pobladas del mundo - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )eswikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"86d6afb4-d73c-49b5-8fc5-16d32323b202","wgCSPNonce":false,"wgCanonicalNamespace":"Anexo","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":104,"wgPageName":"Anexo:Aglomeraciones_urbanas_más_pobladas_del_mundo"

In [6]:
#tabla=soup2.find_all('table', class_='wikitable')

In [7]:
trs=soup2.find_all('tr')

In [8]:
data = []
for tr in trs:
    row = [td.text.replace("\n", "") for td in tr.find_all('td')]
    if (len(row) > 0):
        data.append(row)
df2 = pd.DataFrame(data, columns=['Ranking','Ciudad','Pais', 'Población en último censo', 'Población según citypopulation', 'Población según ONU', 'Población según Demographia', 'Año Censo'])
df2

,Ranking,Ciudad,Pais,Población en último censo,Población según citypopulation,Población según ONU,Población según Demographia,Año Censo
0,1,Cantón,China China,65 565 622,67 800 000,48 638 000,59 259 000,2020
1,2,Tokio,Japón Japón,43.368.953,40.800.000,37 468 000,37 977 000,2020
2,3,Shanghái,China China,41 354 149,40.000.000,25 582 000,24 677 000,2020
3,4,Yakarta,Indonesia Indonesia,33 430 285,33 756 000,17 517 000,34 540 000,2020
4,5,Delhi,India,33 120 876,30 300 000,28 514 000,32 226 000,2011
...,...,...,...,...,...,...,...,...
669,3,Brisbane,Australia,3 150 000,2 338 000,2 180 000,2 054 616,2016
670,4,Perth,Australia,2 075 000,1 991 000,1 978 000,1 874 577,2016
671,5,Auckland,Nueva Zelanda Nueva Zelanda,1 410 000,1 557 000,1 333 000,1 402 020,2019E
672,6,Adelaida,Australia,1 340 000,1 320 000,1 205 000,1 165 639,2016


In [18]:
df2.to_csv('df2.csv', index=False)

Se generó un Data Frame con 8 columnas y 673 filas, que llamamos df2

**Realizamos ahora Web Scraping con Selenium**

**Selenium**

Selenium es un conjunto de herramientas de automatización de pruebas de software que se utilizan para automatizar la interacción de un usuario con una aplicación web. Permite a los desarrolladores y probadores de software crear scripts automatizados para realizar pruebas funcionales en una aplicación web.

In [9]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 4.2 MB/s eta 0:00:00


In [10]:
!pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from logging import error
from selenium.webdriver.support.ui import WebDriverWait

In [17]:
#options = webdriver.ChromeOptions()
#location = '/Users/flore/Desktop/chromedriver'

In [14]:
#options = Options()
#options.add_argument("--window-size=1920x1080")
#options.add_argument("--verbose")

In [16]:
#driver = webdriver.Chrome(options=options, executable_path=location)
#driver.get("https://quotes.toscrape.com")
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
from selenium.webdriver.chrome.service import Service

chrome_path = r"C:/Users/flore/Desktop/chromedriver" # Reemplaza con la ubicación de tu binario de Chrome

service = Service(chrome_path)
driver = webdriver.Chrome(service=service)

##

from webdriver_manager.chrome import ChromeDriverManager

#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--no-sandbox')
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

WebDriverException: ignored

Hacemos el llamado de la url, que es la página que utilizamos con Beautiful Soup: Quotes.toscrape.com

In [ ]:
driver.get(url)


Procedemos a buscar elementos, en este caso Texto y Autor

In [ ]:
#Encontramos elementos

quotes = driver.find_elements(By.CLASS_NAME, "text")
authors = driver.find_elements(By.CLASS_NAME, "author")